# Recomendando Papers - Análisis Exploratorio y Curación de Datos

Arxiv.org participa de la Open Archive Iniciative https://www.openarchives.org/, es decir que provee acceso a toda la metadata de las publicaciones que se suben a su pagina (Leer https://arxiv.org/help/oa).

 En este practico vamos a aprender como acceder a estos datos, limpiarlos y transformarlos en dataframes como los que utilizaron en la materia Análisis y visualización de datos.

## Accediendo a la metadata



La siguiente función nos permite obtener los metadatos de las publicaciones de arxiv.org y devuelve un dataframe de pandas con las siguiente columnas:

*  'title'
* 'id'
* 'abstract'
* 'created'
* 'categories'
* 'doi'
* 'journal'

Esta función tiene como parámetros **arxiv_set** que indica que set de publicaciones vamos a llamar (math, cs, physics, etc, http://export.arxiv.org/oai2?verb=ListSets)  y **year** que determina el año de las publicaciones. (Tambien puede modificarse facilmente para extraer metadata de fechas determinadas)
 

---



In [0]:
import time
import urllib
import datetime
#from itertools import ifilter
from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
import urllib.error,urllib.request

from bs4 import BeautifulSoup
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
#import bibtexparser

pd.set_option('mode.chained_assignment','warn')

In [0]:
OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

def harvest(arxiv_set="math",year=2018):
    df = pd.DataFrame(columns=("title", "abstract", "categories", "created", "id", "doi",'authors','journal'))
    base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"
    url = (base_url +
           "from={}-01-01&until={}-12-31&".format(year,year) +
           "metadataPrefix=arXiv&set=%s"%arxiv_set)
    
    while True:
        print("fetching", url)
        try:
            response = urllib.request.urlopen(url)
            
        except urllib.error.HTTPError as e:
            if e.code == 503:
                to = int(e.hdrs.get("retry-after", 30))
                print("Got 503. Retrying after {0:d} seconds.".format(to))

                time.sleep(to)
                continue
                
            else:
                raise
            
        xml = response.read()

        root = ET.fromstring(xml)

        for record in root.find(OAI+'ListRecords').findall(OAI+"record"):
            arxiv_id = record.find(OAI+'header').find(OAI+'identifier')
            meta = record.find(OAI+'metadata')
            info = meta.find(ARXIV+"arXiv")
            created = info.find(ARXIV+"created").text
            created = datetime.datetime.strptime(created, "%Y-%m-%d")
            categories = info.find(ARXIV+"categories").text
            
            try:
              journal=info.find(ARXIV+"journal-ref").text
            except AttributeError:
              journal=None
            
            authors=info.find(ARXIV+"authors")
            authors_list=[]
            
            for author in authors.findall(ARXIV + 'author'):
                 try: 
                  fullname= author.find(ARXIV + 'keyname').text + ' ' + author.find(ARXIV + 'forenames').text
                 except:
                  try: 
                   fullname=author.find(ARXIV + 'forenames').text
                  except:
                   fullname=author.find(ARXIV + 'keyname').text  
                 authors_list.append(fullname)

                
            doi = info.find(ARXIV+"doi")
            if doi is not None:
                doi = doi.text.split()[0]
                
            contents = {'title': info.find(ARXIV+"title").text,
                        'id': info.find(ARXIV+"id").text,#arxiv_id.text[4:],
                        'abstract': info.find(ARXIV+"abstract").text.strip(),
                        'created': created,
                        'categories': ",".join(categories.split()),
                        'doi': doi,
                        'authors': ",".join(authors_list),
                        'journal' : journal
                        }

            df = df.append(contents, ignore_index=True)

        # The list of articles returned by the API comes in chunks of
        # 1000 articles. The presence of a resumptionToken tells us that
        # there is more to be fetched.
        token = root.find(OAI+'ListRecords').find(OAI+"resumptionToken")
        if token is None or token.text is None:
            break

        else:
            url = base_url + "resumptionToken=%s"%(token.text)
            
    return df
    

1 -    Usen la función **harvest()** definida previamente para crear 3 dataframes correspondientes a cada set "math", "cs" y "physics", para el año 2019 y guarden cada uno de ellos como un archivo csv para evitar tener que usar la api de arxiv innecesariamente. (y en los siguientes puntos pueden/deben guardar nuevas versiones del dataset cuando les parezca conveniente para evitar tener que correr procesos nuevamente)







In [0]:
df_math=harvest('math',year=2019)

fetching http://export.arxiv.org/oai2?verb=ListRecords&from=2019-01-01&until=2019-12-31&metadataPrefix=arXiv&set=math
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|1001
Got 503. Retrying after 10 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|1001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|2001
Got 503. Retrying after 10 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|2001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|3001
Got 503. Retrying after 10 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|3001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|4001
Got 503. Retrying after 10 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=3634276|4001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionT

2 - A continuación carguen los archivos csv como dataframes de pandas nuevamente y corroboren que los datos esten bien cargados, los tipos de cada columna sean los correctos, que no haya columnas nulas, etc. 
Además revisen que no haya datos  duplicados (¿podria haber  publicaciones con el mismo titulo? ¿con el mismo abstract? ¿con el mismo id?). Y checkeen que las fechas de cada fila correspondan realmente al año 2019, si existen datos de otros años eliminenlos (¿porque podria pasar esto?¿Cual es la ultima fecha que aparece en el dataset? )

3 -  Ahora deben juntar los 3 dataframes en uno solo **df_arxiv_2019** y volver a revisar si hay filas duplicadas. Capitalizar los nombres de las columnas (title --> Title)

4 - Agregar al dataframe **df_arxiv_2019** columnas nuevas correspondientes a "Year", "Month" y "Day" a partir de la columna "Created"

5 - Crear nuevas columnas 'Math', 'Cs' , 'Physics' con valores 0,1 correspondientes a si la publicación es parte de cada uno de las areas (sets). 

6 -  Agregar las columnas faltantes
* Abstract_Length [int64] : Cantidad de caracteres en el abstract.

* Title_Length [int64] : Cantidad de caracteres en el título.

* Number_Authors [ int64] : Cantidad de autores.

* Number_Fields [int64] : Número de Áreas.

* Number_Categories [int64]: Número de Categorias.